In [ ]:
#| hide
from ITS_LIVE_Analysis.core import *

# ITS_LIVE_Analysis

> Repository gathering useful notebooks and code to work with ITS_LIVE Data

This file will become your README and also the index of your documentation.

## Install

```sh
pip install ITS_LIVE_Analysis
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2